BERIKUT UTK DOWNGRADE KE PYTHON 3.8.0

In [1]:
!which python
!python --version

/usr/local/bin/python
Python 3.11.13


In [2]:
%env PYTHONPATH=

env: PYTHONPATH=


In [3]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 33.1 MB/s eta 0:00:00


In [4]:
!virtualenv myenv

created virtual environment CPython3.11.13.final.0-64 in 570ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [5]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh

--2025-07-12 16:29:41--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘Miniconda3-py38_4.8.2-Linux-x86_64.sh’

Miniconda3-py38_4.8 100%[===================>]  85.66M   238MB/s    in 0.4s    

2025-07-12 16:29:42 (238 MB/s) - ‘Miniconda3-py38_4.8.2-Linux-x86_64.sh’ saved [89817099/89817099]



In [6]:
!chmod +x Miniconda3-py38_4.8.2-Linux-x86_64.sh

In [7]:
!./Miniconda3-py38_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8=py38h1ba5d50_0
    - idna==2.8=py38_1000
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.2.1=hd88cf55_4
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_0
    - openssl==1.1.1d=h7b6447c_4
    - pip==20.0.2=py38_1
    - pycosat==0.6.3=py38h7b6447c_0
    - pycparser==2.19=py_0
    - pyopenssl==19.1.0=py38_0
    - pysocks==1.7.1=py38_0
    - python==3.8.1=h0371630_1
    - readline==7.0=h7b6447c_5
    - requests==2.22.0=py38_1
    - ruamel_yaml==0.15.

In [8]:
!conda install -q -y --prefix /usr/local python=3.8 ujson

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - python=3.8
    - ujson


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.2.25  |       h06a4308_0         129 KB
    certifi-2020.6.20          |     pyhd3eb1b0_3         155 KB
    openssl-1.1.1w             |       h7f8727e_0         3.7 MB
    ujson-1.35                 |   py38h7b6447c_0          34 KB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  ujson              pkgs/main/linux-64::ujson-1.35-py38h7b6447c_0

The following packages will be UPDATED:

  ca-certificates                                2020.1.1-0 --> 2025.2.25-h06a4308_0
  certifi            pkgs/main/linux-64::certifi-2019.11.2~ --> pkgs/ma

In [9]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')

In [10]:
import os
os.environ['CONDA_PREFIX'] = '/usr/local/envs/myenv'

In [11]:
!python --version

Python 3.8.1


BERIKUT PERSIAPAN TRAINING TENSORFLOW LITE

In [12]:
!pip uninstall Cython -y # Temporary fix for "No module named 'object_detection'" error
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4353, done.
remote: Counting objects: 100% (4353/4353), done.
remote: Compressing objects: 100% (3162/3162), done.
remote: Total 4353 (delta 1188), reused 3992 (delta 1118), pack-reused 0 (from 0)
Receiving objects: 100% (4353/4353), 56.07 MiB | 20.54 MiB/s, done.
Resolving deltas: 100% (1188/1188), done.


In [13]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [14]:
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [15]:
!pip install pyyaml==5.3

     |████████████████████████████████| 268 kB 545 kB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3-cp38-cp38-linux_x86_64.whl size=603705 sha256=06ca30a0ac96c27c63695f2f90f7600e2a55975d43a97966c4c469815a4fe565
  Stored in directory: /root/.cache/pip/wheels/e4/94/ec/65e430f66fda7a86c08f61ce59455581adc269a65881b4ffeb
Successfully built pyyaml


In [16]:
!pip install /content/models/research/

Processing ./models/research
     |████████████████████████████████| 16.2 MB 307 kB/s 
     |████████████████████████████████| 4.4 MB 29.5 MB/s 
     |████████████████████████████████| 5.1 MB 43.7 MB/s 
     |████████████████████████████████| 9.2 MB 56.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
     |████████████████████████████████| 352 kB 54.8 MB/s 
     |████████████████████████████████| 439 kB 47.6 MB/s 
     |████████████████████████████████| 34.5 MB 1.2 MB/s 
     |████████████████████████████████| 12.4 MB 53.8 MB/s 
     |████████████████████████████████| 2.2 MB 30.0 MB/s 
     |████████████████████████████████| 28.8 MB 426 kB/s 
     |████████████████████████████████| 1.7 MB 66.1 MB/s 
     |████████████████████████████████| 67 kB 8.0 MB/s 
     |████████████████████████████████| 116 kB 64.6 MB/s 
     |████████████████████████████████| 130 kB 63.9 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
     |████████████████████████████████| 3.2 MB

In [1]:
!pip install tensorflow==2.8.0

     |████████████████████████████████| 497.6 MB 1.6 kB/s 
     |████████████████████████████████| 462 kB 53.8 MB/s 
     |████████████████████████████████| 1.4 MB 40.6 MB/s 
ERROR: tf-keras 2.15.1 has requirement tensorflow<2.16,>=2.15, but you'll have tensorflow 2.8.0 which is incompatible.
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.4
    Uninstalling tensorflow-2.8.4:
      Successfully uninstalled tensorflow-2.8.4


In [2]:
!pip install tensorflow_io==0.23.1

     |████████████████████████████████| 23.1 MB 468 kB/s 
     |████████████████████████████████| 2.1 MB 145 kB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.34.0
    Uninstalling tensorflow-io-gcs-filesystem-0.34.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.34.0
  Attempting uninstall: tensorflow-io
    Found existing installation: tensorflow-io 0.34.0
    Uninstalling tensorflow-io-0.34.0:
      Successfully uninstalled tensorflow-io-0.34.0


In [3]:
!sudo apt-get remove -y protobuf-compiler

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  libprotoc23
Use 'sudo apt autoremove' to remove it.
The following packages will be REMOVED:
  protobuf-compiler
0 upgraded, 0 newly installed, 1 to remove and 35 not upgraded.
After this operation, 113 kB disk space will be freed.
(Reading database ... 126281 files and directories currently installed.)
Removing protobuf-compiler (3.12.4-1ubuntu7.22.04.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [4]:
!pip install 'protobuf<=3.20.1' --force-reinstall

     |████████████████████████████████| 1.0 MB 782 kB/s 
ERROR: tensorflow-metadata 1.14.0 has requirement absl-py<2.0.0,>=0.9, but you'll have absl-py 2.3.1 which is incompatible.
ERROR: tensorflow-metadata 1.14.0 has requirement protobuf<4.21,>=3.20.3, but you'll have protobuf 3.20.1 which is incompatible.
ERROR: googleapis-common-protos 1.70.0 has requirement protobuf!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you'll have protobuf 3.20.1 which is incompatible.
ERROR: google-api-core 2.25.1 has requirement protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.19.5, but you'll have protobuf 3.20.1 which is incompatible.
ERROR: apache-beam 2.60.0 has requirement protobuf!=4.0.*,!=4.21.*,!=4.22.0,!=4.23.*,!=4.24.*,<4.26.0,>=3.20.3, but you'll have protobuf 3.20.1 which is incompatible.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalle

In [1]:
!wget https://github.com/protocolbuffers/protobuf/releases/download/v3.20.1/protoc-3.20.1-linux-x86_64.zip

--2025-07-12 16:41:09--  https://github.com/protocolbuffers/protobuf/releases/download/v3.20.1/protoc-3.20.1-linux-x86_64.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/23357588/b810fab9-307d-4294-a3b1-fa0555dc22c3?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-12T17%3A32%3A22Z&rscd=attachment%3B+filename%3Dprotoc-3.20.1-linux-x86_64.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-12T16%3A32%3A11Z&ske=2025-07-12T17%3A32%3A22Z&sks=b&skv=2018-11-09&sig=nVnaCzaQtHIoCzvXVzCQVOmsZN9G4qJXpFZ3aCDvYCg%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1MjMzODc2OSwibmJmIjoxNzUyMzM4NDY5LCJwYXRoI

In [2]:
!unzip protoc-3.20.1-linux-x86_64.zip -d protoc3

Archive:  protoc-3.20.1-linux-x86_64.zip
   creating: protoc3/include/
   creating: protoc3/include/google/
   creating: protoc3/include/google/protobuf/
  inflating: protoc3/include/google/protobuf/any.proto  
  inflating: protoc3/include/google/protobuf/descriptor.proto  
  inflating: protoc3/include/google/protobuf/duration.proto  
  inflating: protoc3/include/google/protobuf/api.proto  
  inflating: protoc3/include/google/protobuf/source_context.proto  
  inflating: protoc3/include/google/protobuf/wrappers.proto  
  inflating: protoc3/include/google/protobuf/struct.proto  
   creating: protoc3/include/google/protobuf/compiler/
  inflating: protoc3/include/google/protobuf/compiler/plugin.proto  
  inflating: protoc3/include/google/protobuf/field_mask.proto  
  inflating: protoc3/include/google/protobuf/timestamp.proto  
  inflating: protoc3/include/google/protobuf/empty.proto  
  inflating: protoc3/include/google/protobuf/type.proto  
   creating: protoc3/bin/
  inflating: protoc3/b

In [3]:
!sudo mv protoc3/bin/* /usr/local/bin/

In [4]:
!sudo mv protoc3/include/* /usr/local/include

In [ ]:
# Install CUDA version 11.0 (to maintain compatibility with TF v2.8.0)
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install cuda-toolkit-11-0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

In [5]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2025-07-12 16:41:32.788253: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-07-12 16:41:32.788284: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-07-12 16:41:38.626840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-07-12 16:41:38.626885: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-07-12 16:41:38.626916: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this 

In [6]:
!mkdir /content/images
!unzip -q images.zip -d /content/images/all
!mkdir /content/images/train; mkdir /content/images/validation; mkdir /content/images/test